# Save Exposures of postISRCCD in fits file (at CC)


- work with Weakly_2022_39
- use jupyter kernel LSST
- kernel lsst_distrib_w_2022_39


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/03/22

- update : 2022/05/13
- last update : 2022/11/08 (remove rererences for plotting)




In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/drp_pipe 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_39/conda/envs/lsst-scipipe-4.1.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_39/rapid_analysis 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g0b29ad24fb+a10408d0bf 	w_latest current w_2022_39 setup


In [5]:
import lsst.daf.butler as dafButler

In [3]:
import os

import numpy as np
import pandas as pd


from astropy.io import fits

# Configuration

## Butler

In [6]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

## Utils

In [7]:
def isflat(row):
    #print(row["filter"])
    if str(row["type"]) == "flat":
        return True
    
    else:
        return False 

## Dataset Query

In [8]:
collection = "u/dagoret/first_test2"
datasetRefs = registry.queryDatasets(datasetType='postISRCCD', collections=collection, where= "instrument='LATISS'")

In [9]:
pd.set_option('display.max_rows', 2000)
column_names = ["id","run","type_name","band","filter","exposure"]
df = pd.DataFrame(columns = column_names)

In [10]:
all_ref_for_postisrccd = []

#for i, ref in enumerate(datasetRefs):
#    all_ref_for_postisrccd.append(ref.dataId["exposure"])
#    print("fullId: ",ref.dataId.full)
    
for count, ref in enumerate(datasetRefs):    
    all_ref_for_postisrccd.append(ref.dataId["exposure"])

    row = [ref.id ,
           ref.run,
           ref.datasetType.name,
           ref.dataId["band"],
           ref.dataId["physical_filter"],
           ref.dataId["exposure"] ]
    #print(row)
    df.loc[count] = row
    
 

In [11]:
all_ref_for_postisrccd=np.array(all_ref_for_postisrccd)

In [12]:
dates_of_exposures = (all_ref_for_postisrccd/100000).astype(int)

In [13]:
#dates_of_exposures

In [14]:
df["exposure_date"] = (df["exposure"]//100000).astype(int)

In [15]:
df["expseq"] = df["exposure"] - df["exposure_date"]*100000

In [16]:
# too many
#df

In [17]:
df.columns

Index(['id', 'run', 'type_name', 'band', 'filter', 'exposure', 'exposure_date',
       'expseq'],
      dtype='object')

In [18]:
df["exposure_date"].unique()

array([20220317, 20220216, 20220316, 20211104, 20220215, 20210707,
       20220607, 20220608, 20220609, 20211103, 20211102, 20220629,
       20220630, 20210909, 20220628])

In [19]:
df["filter"].unique()

array(['empty~ronchi170lpmm', 'empty~holo4_003', 'BG40~ronchi170lpmm',
       'FELH0600~holo4_003', 'FELH0600~ronchi170lpmm', 'BG40~holo4_003',
       'SDSSg~holo4_003', 'SDSSg~ronchi170lpmm'], dtype=object)

# Selection of date and filter

In [20]:
#DATESEL =  20210909
#DATESEL =  20211104
#DATESEL =  20220630
DATESEL =  20211103

In [21]:
all_filters_thiday = df[df["exposure_date"]== DATESEL]["filter"].unique()
all_filters_thiday 

array(['BG40~holo4_003', 'FELH0600~holo4_003', 'empty~holo4_003'],
      dtype=object)

In [22]:
len(all_filters_thiday) 

3

In [23]:
#FILTERSEL = "empty~ronchi170lpmm"
#FILTERSEL = 'empty~holo4_003'
FILTERSEL = all_filters_thiday[2] 

In [25]:
cut = (df["exposure_date"] == DATESEL) &  (df["filter"] == FILTERSEL)

In [26]:
df_sel = df[cut]

In [27]:
df_sel

,id,run,type_name,band,filter,exposure,exposure_date,expseq
2680,7a77a831-00ef-4d5a-8ea2-102875a90644,u/dagoret/first_test2/20221108T130917Z,postISRCCD,white,empty~holo4_003,2021110300709,20211103,709
2694,3aae72dc-57b2-4d80-9a67-779463dbef22,u/dagoret/first_test2/20221108T130919Z,postISRCCD,white,empty~holo4_003,2021110300721,20211103,721
2696,d5c389b6-d41f-45aa-85db-a420d30cd2ec,u/dagoret/first_test2/20221108T143624Z,postISRCCD,white,empty~holo4_003,2021110300574,20211103,574
2700,9597be9b-d387-4649-ac91-8b98535089a6,u/dagoret/first_test2/20221108T131020Z,postISRCCD,white,empty~holo4_003,2021110300495,20211103,495
2704,c47dbb21-168b-4f9a-884e-0df3d85a6841,u/dagoret/first_test2/20221108T131021Z,postISRCCD,white,empty~holo4_003,2021110300733,20211103,733
2708,a37100c0-3778-4487-8a92-2d21e18bffc3,u/dagoret/first_test2/20221108T143711Z,postISRCCD,white,empty~holo4_003,2021110300580,20211103,580
2711,32b93f88-607c-4f36-acab-b7127645672a,u/dagoret/first_test2/20221108T131022Z,postISRCCD,white,empty~holo4_003,2021110300697,20211103,697
2715,5c6c7209-2691-48f0-aadb-8474f1518239,u/dagoret/first_test2/20221108T143754Z,postISRCCD,white,empty~holo4_003,2021110300586,20211103,586
2717,3a0f3ece-d8e8-4f3e-945d-ae5382356d57,u/dagoret/first_test2/20221108T131024Z,postISRCCD,white,empty~holo4_003,2021110300513,20211103,513
2720,640255e6-4ce1-4053-8894-ac46688646af,u/dagoret/first_test2/20221108T131025Z,postISRCCD,white,empty~holo4_003,2021110300586,20211103,586


In [28]:
all_exposures_selected = df_sel["exposure"].values

In [29]:
all_exposures_selected

array([2021110300709, 2021110300721, 2021110300574, 2021110300495,
       2021110300733, 2021110300580, 2021110300697, 2021110300586,
       2021110300513, 2021110300586, 2021110300465, 2021110300592,
       2021110300537, 2021110300598, 2021110300543, 2021110300604,
       2021110300525, 2021110300580, 2021110300610, 2021110300459,
       2021110300691, 2021110300465, 2021110300697, 2021110300471,
       2021110300703, 2021110300477, 2021110300709, 2021110300483,
       2021110300715, 2021110300489, 2021110300721, 2021110300495,
       2021110300727, 2021110300501, 2021110300733, 2021110300507,
       2021110300513, 2021110300519, 2021110300525, 2021110300531,
       2021110300537, 2021110300543, 2021110300556, 2021110300562,
       2021110300568])

# loop on exposures to produce fits files and save fits file

In [30]:
from pathlib import Path

In [31]:
# Where all output image goes (year is related to the release, not the observation date
topoutputdir = "/sps/lsst/groups/auxtel/data/2022"
outputdir = os.path.join(topoutputdir,FILTERSEL)
outputsubdir = os.path.join(outputdir,str(DATESEL))

In [32]:
outputsubdir

'/sps/lsst/groups/auxtel/data/2022/empty~holo4_003/20211103'

In [33]:
Path(outputsubdir).mkdir(parents=True, exist_ok=True)

In [ ]:
for exposure_selected in all_exposures_selected:
    
    raw= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
    meta = raw.getMetadata()
    md = meta.toDict()
    
    md['HA'] = (md['HASTART']+ md['HAEND'])/2.
    
    targetname = md['OBJECT']
    targetname_split=targetname.split(":")
    print("--- ",targetname)
    if len(targetname_split)==2:
        targetname=targetname_split[-1]
        md['OBJECT']=targetname
        if targetname == "ETADOR" or targetname == "ETA1DOR" or targetname == "ETA2DOR":
            targetname="eta dor"
        if targetname == "MUCOL":
            targetname="mu. col"
        
    print(targetname)
    md['OBJECT']=targetname
    
    ccd = butler.get('postISRCCD', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
    img_data =ccd.image.array
    
    filenameout = "exposure_" + str(exposure_selected)+"_postisrccd.fits"
    fullfilenameout = os.path.join(outputsubdir,filenameout)
    
    #flip_image=np.flip(img_data, 0)  # flip the image for Spectractor
    flip_image = np.flipud(np.fliplr(img_data))
    
    
    hdr = fits.Header()
    
    for key,value in md.items():
        hdr[str(key)]=value
        
    primary_hdu = fits.PrimaryHDU(header=hdr)
    image_hdu = fits.ImageHDU(flip_image)
    hdu_list = fits.HDUList([primary_hdu, image_hdu])
    
    hdu_list.writeto(fullfilenameout ,overwrite=True)
    
    
print("This is the End !")    

In [ ]:
print("hello")